In [2]:
import pandas as pd 
import os
import requests
import json

/Users/vivek/Drive E/Hackathons/Stylumia/stylumia-nxt/style_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
data_directory = 'data'
categories = []
data_coloumns = []
data = {}


#add sno column to data
# for filename in os.listdir(data_directory):
#     if os.path.isdir(os.path.join(data_directory, filename)):
#         continue
#     if not filename.endswith('.csv'):
#         continue
#     temp_data = pd.read_csv(os.path.join(data_directory, filename))
#     l = len(temp_data)
#     temp_data.insert(0, 'sno', range(1, l + 1))
#     temp_data.to_csv(os.path.join(data_directory, filename), index=False)
    

for filename in os.listdir(data_directory):
    if os.path.isdir(os.path.join(data_directory, filename)):
        continue
    if not filename.endswith('.csv'):
        continue
    cat = filename.split()[0]
    print(f'category:{cat}')
    categories.append(cat)
    data[cat] = pd.read_csv(os.path.join(data_directory, filename))


category:Kurtis
category:Dresses
category:Saree
category:Shirts
category:Bathroom
category:Watches
category:Jeans
category:Sneakers
category:Earrings
category:Tshirts


In [5]:
data_coloumns = data[categories[0]].columns
usefull_columns = ['sno', 'product_id', 'product_name', 'description', 'meta_info', 'brand','feature_list', 'style_attributes']

for c in data_coloumns:
    print(f"Column: {c}")
print("\n")
for c in usefull_columns:
    print(f"Usefull Column: {c}")



Column: sno
Column: description
Column: category_id
Column: meta_info
Column: product_id
Column: pdp_url
Column: sku
Column: brand
Column: feature_image
Column: department_id
Column: mrp
Column: product_name
Column: feature_image_s3
Column: channel_id
Column: feature_list
Column: style_attributes
Column: category_name
Column: Retailer_name


Usefull Column: sno
Usefull Column: product_id
Usefull Column: product_name
Usefull Column: description
Usefull Column: meta_info
Usefull Column: brand
Usefull Column: feature_list
Usefull Column: style_attributes


# CSV Processing

In [6]:
#make new csv files from the "data" such that it only contains the usefull columns
for cat, cat_data in data.items():
    cat_data = cat_data[usefull_columns]
    cat_data.to_csv(f'data/processed_csv/{cat}.csv', index=False)

# EDA

In [30]:
#append category in front in usefull_comumns
temp_columns = usefull_columns.copy()
temp_columns.insert(0, 'num_rows')
temp_columns.insert(0, 'category')

df = pd.DataFrame(columns=temp_columns)
    
for cat in categories:
    num_rows = len(data[cat])
    print(cat)
    temp_dict = {
        'category': cat,
        'num_rows': num_rows,
        'description': len(data[cat][data[cat]['description'].isnull()]) + len(data[cat][data[cat]['description'] == '']),
        'meta_info': len(data[cat][data[cat]['meta_info'].isnull()]) + len(data[cat][data[cat]['meta_info'] == '']),
        'brand': len(data[cat][data[cat]['brand'].isnull()]) + len(data[cat][data[cat]['brand'] == '']),
        'product_name': len(data[cat][data[cat]['product_name'].isnull()]) + len(data[cat][data[cat]['product_name'] == '']),
        'feature_list': len(data[cat][data[cat]['feature_list'] == '[]']) + len(data[cat][data[cat]['feature_list'].isnull()]),
        'style_attributes': len(data[cat][data[cat]['style_attributes'].eq("{}")]) + len(data[cat][data[cat]['style_attributes'].isnull()]),
        'category_name': len(data[cat][data[cat]['category_name'].isnull()]) + len(data[cat][data[cat]['category_name'] == '']),
        'Retailer_name': len(data[cat][data[cat]['Retailer_name'].isnull()] + len(data[cat][data[cat]['Retailer_name'] == '']))
    }
    df = pd.concat([df, pd.DataFrame([temp_dict])])

df

Kurtis
Dresses
Saree
Shirts
Bathroom
Watches
Jeans
Sneakers
Earrings
Tshirts


,category,num_rows,description,meta_info,brand,product_name,feature_list,style_attributes,category_name,Retailer_name
0,Kurtis,14031,0,0,0,0,14031,0,0,0
0,Dresses,7795,6889,4,0,0,7795,4,0,0
0,Saree,7305,0,0,0,0,7264,41,0,0
0,Shirts,9106,8830,0,0,0,9106,0,0,0
0,Bathroom,9546,0,0,0,0,441,0,0,0
0,Watches,9856,2955,0,0,0,1871,6,0,0
0,Jeans,21366,21082,0,0,0,21366,0,0,0
0,Sneakers,4660,4660,0,0,0,0,0,0,0
0,Earrings,5112,5041,0,0,0,71,0,0,0
0,Tshirts,5721,5665,0,0,0,5721,0,0,0


In [15]:
import re

str = "['Colour:Pink', 'Fabric:Organza', 'Items Included:Saree', 'Wash Care:Dry Clean Only', 'Disclaimer Text:Product color may slightly vary due to photographic lighting sources or your monitor/screen settings.', 'Manufacturer:Vedant Fashions Ltd, Paridhan Garment Park, 19, Canal South Road, SDF1, 4th Floor, A501-A502, Kolkata - 700015, West Bengal.']"
# Extract text before ':' and after "'" or '"'
matches = re.findall(r"'([^']*?):", str)
matches = re.findall(r":([^']*?)'", str)

matches


# i = str
# i = i.replace("'", '"')
# i = i.replace("[", "{")
# i = i.replace("]", "}")
# print(i)
# d = json.loads(i.replace("'", '"'))
# d

['Pink',
 'Organza',
 'Saree',
 'Dry Clean Only',
 'Product color may slightly vary due to photographic lighting sources or your monitor/screen settings.',
 'Vedant Fashions Ltd, Paridhan Garment Park, 19, Canal South Road, SDF1, 4th Floor, A501-A502, Kolkata - 700015, West Bengal.']

In [16]:
def extract_features_using_regex(feature_list):
    features = set()
    for i in feature_list:
        if i == '[]':
            continue

        a = re.findall(r"'([^']*?)\:", i)
        b = re.findall(r"\:([^']*?)'", i)
        d = dict(zip(a, b))
        for k, v in d.items():
            if k == '':
                continue
            features.add(k)

        a = re.findall(r"\"([^']*?)\:", i)
        b = re.findall(r"\:([^']*?)\"", i)
        d = dict(zip(a, b))
        for k, v in d.items():
            if k == '':
                continue
            features.add(k)
    return features

def extract_features_using_json(feature_list):
    features = set()
    for i in feature_list:
        if i == '[]':
            continue
        try:
            i = i.replace("'", '"')
            i = i.replace("[", "{")
            i = i.replace("]", "}")
            d = json.loads(i)
        except:
            continue
        for k, v in d.items():
            features.add(k)
    return features

In [17]:
for cat in categories:
    print(f"Category: {cat}")

    feature_list = data[cat]['feature_list']
    style_attributes = data[cat]['style_attributes']

    #convert the string into dict
    
    # feature_list_transformed = feature_list.apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('{') and x.endswith('}') else x)
    # style_attributes_transformed = style_attributes.apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('{') and x.endswith('}') else x)

    # feature_list = feature_list.apply(lambda x: eval(x))
    # style_attributes = style_attributes.apply(lambda x: eval(x))

    features1 = extract_features_using_regex(feature_list)
    attributes1 = extract_features_using_regex(style_attributes)

    features2 = extract_features_using_json(feature_list)
    attributes2 = extract_features_using_json(style_attributes)

    features = features1.union(features2)
    attributes = attributes1.union(attributes2)
            

    print(f"Features: {features}")
    print(f"Attributes: {attributes}")        
    
    print("\n")

Category: Kurtis
Features: set()
Attributes: {'GPS', 'Supplier Information', 'WORLD'}


Category: Dresses
Features: set()
Attributes: {'Concept', 'Top', 'Outerwear Composition', 'Net Quantity', 'Care Instructions', 'Fabric', 'Dresses Composition', 'Features', 'Collection', 'Pattern Type', 'Fit', 'Size', 'Details', 'Dresses Length', 'Sleeve', 'Slit', 'Date of manufacture', 'Sheer', 'Skirts Material', 'Neckline', 'Date of import', 'Common generic name', 'Waist Line', 'Body', 'Women Panties Composition', 'Bottom part', 'Fit Type', 'Dresses Lining', 'Dresses Sleeve Length', 'Material', 'Sleeve Type', 'Number of Pieces', 'Coating', 'Price (MRP)', 'Belt', 'Outerwear Length', 'Nice to know', 'Headwear Material', 'Gloves Material', 'Occasion', 'Description', 'Country of production', 'Length', 'Special Features', 'Sports activity', 'Chest pad', 'XXS ', 'Dresses Body', 'Upper', 'Color', 'Care instructions', 'Skirts Composition', 'Tops Length', 'Tops Sleeve Length', 'Front', 'Functional Type', 'C

In [34]:
for c in usefull_columns:
    print(f"{c}: {data['Saree'][c][100]}")

description: This exquisite silk sari features a stunning woven pattern, perfect for any celebration. The luxurious fabric drapes beautifully, adding elegance to your festive look. Pair it with traditional gold jewelry for a timeless appeal. Care: dry clean only.
meta_info: The model's height is 5' 6 Size & Fit FABINDIA LIMITED.Plot No. 10, Local Shopping Complex, Sector B, Pocket- 7, Vasant Kunj, New Delhi- 110070,PH: 011-40692000,Email: mailus@fabindia.net Manufactured & Marketed By Dry Clean Only Care Instructions India Country Of Origin 6.4M X1.16M (inclusive of blouse piece) Dimensions Silk Material
brand: fabindia
product_name: Orange Silk Woven Sari
feature_list: []
style_attributes: {'Size & Fit': "The model's height is 5' 6", 'Manufactured & Marketed By': 'FABINDIA LIMITED.Plot No. 10, Local Shopping Complex, Sector B, Pocket- 7, Vasant Kunj, New Delhi- 110070,PH: 011-40692000,Email: mailus@fabindia.net', 'Care Instructions': 'Dry Clean Only', 'Country Of Origin': 'India', 'Di

# Image Downloads

In [ ]:
for cat, cat_data in data.items():
    if(cat != 'Kurtis'):
        continue
    print(f"Category: {cat}, Data Shape: {cat_data.shape}")
    for i in range(40):
        if os.path.exists(f'data/processed_image/{cat}/{i}.jpg'):
            continue

        response = requests.get(cat_data.loc[i, 'feature_image_s3'])
        if response.status_code == 200:
            with open(f'data/processed_image/{cat}/{i}.jpg', 'wb') as file:
                file.write(response.content)
            print("Image downloaded successfully.")
        else:
            print("Failed to download image.")
